In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# from google.colab import runtime
# runtime.unassign()

In [6]:
%cd "/content/drive/MyDrive/Colab Notebooks/nnHW4"

/content/drive/MyDrive/Colab Notebooks/nnHW4


In [7]:
# !wget http://csr.bu.edu/ftp/visda/2019/multi-source/real.zip

In [8]:
# !ls

In [19]:
from typing import Dict, Tuple, List
import zipfile


In [10]:
mammals = {
    'name': 'mammals',
    'list_to_extract': [
        'cat',
        'hedgehog',
        'lion',
        'panda',
        'raccoon',
        'squirrel',
        'zebra',
        'bear',
        'cow',
        'elephant',
        'horse',
        'monkey',
        'pig',
        'rhinoceros',
        'tiger',
        'camel',
        'dog',
        'giraffe',
        'kangaroo',
        'mouse',
        'rabbit',
        'sheep',
    ]
}

road_transportation = {
    'name': 'road_transportation',
    'list_to_extract': [
        'ambulance',
        'bus',
        'motorbike',
        'roller_coaster',
        'train',
        'bicycle',
        'car',
        'pickup_truck',
        'school_bus',
        'truck',
        'bulldozer',
        'firetruck',
        'police_car',
        'tractor',
        'van',
    ]
}

In [21]:
def check_classes():
    zip_ref = zipfile.ZipFile(f"/content/drive/MyDrive/Colab Notebooks/nnHW4/real.zip", 'r')
    namelist=zip_ref.namelist()

    list_to_extract = ['mouse']
    name_to_extract = ['mouse']
    filtered_namelist = [string for string in namelist if any(substring in string for substring in list_to_extract)]
    print(len(filtered_namelist))

check_classes()

148


In [22]:
def unzip_classes(what_to_extract: Dict, domain: str):
    zip_ref = zipfile.ZipFile(f"/content/drive/MyDrive/Colab Notebooks/nnHW4/{domain}.zip", 'r')
    namelist=zip_ref.namelist()
    list_to_extract = what_to_extract['list_to_extract']
    name_to_extract = what_to_extract['name']
    filtered_namelist = [string for string in namelist if any(substring in string for substring in list_to_extract)]
    print(len(filtered_namelist))
    # zip_ref.extractall(path=f'DomainNet/{domain}/{name_to_extract}', members = filtered_namelist)

In [23]:
# unzip_classes(mammals, 'real')

15850


In [13]:
from torchvision.datasets import ImageFolder
import torchvision
import torch
import torch.nn as nn
from torchvision import datasets
import numpy as np
import os
import random
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
import os
from torchvision.datasets.folder import default_loader

In [28]:
import matplotlib.pyplot as plt
from tqdm import tqdm

def get_samples(dataloader, num_samples: int, images_are_normalized: bool, num_classes) -> np.ndarray:
    samples = []
    for _ in range(num_classes):
        samples.append([])

    for (images, labels) in tqdm(dataloader):
        # images are normalized using mean=(0.5, 0.5, 0.5) and std=(0.5, 0.5, 0.5),
        # so images habve been normalized using: image = image - mean / std
        # to plot images we have to undo the normalization
        if images_are_normalized:
            images = images * 0.5 + 0.5

        # print(torch.unique(labels, return_counts=True))

        for img_indx, curr_image in enumerate(images):
            if len(samples[labels[img_indx]]) < num_samples:
                samples[labels[img_indx]].append(curr_image.numpy())

        added = False
        for i in range(num_classes):
            if len(samples[i]) < num_samples:
                added = True
                break

        if not added:
            break


    # convert samples to numpy array
    return np.array(samples)

import os
def make_dir(dir_name: str):
    """
    creates directory "dir_name" if it doesn't exists
    """
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

def plot_samples(samples, classes_str, images_are_normalized: bool, path: str, filename: str = 'samples', path_to_save: str = 'exploring_data', phase: str = 'train'):
    make_dir(path)

    for class_indx in range(samples.shape[0]):
        for sample_indx in range(samples.shape[1]):
                image = np.transpose(samples[class_indx][sample_indx], (1, 2, 0))  # Transpose the image to (32, 32, 3)
                plt.imshow(image)
                plt.savefig(f'{path}/{classes_str[class_indx]}_{class_indx} sample_{sample_indx}.jpg')
                plt.clf()

In [29]:

def get_domain_net(domain: str,
                   group: str,
                   normalize_data: bool,\
                   image_size: int,
                   batch_size=128,
                   randseed: int =11, pin_memory=False,
                   num_workers=1):

    assert domain == 'real' or domain == 'painting', 'only real and painting domain.'
    assert group == 'mammals', 'only mammals group of dataset'

    def seed_worker(worker_id):
        # worker_seed = torch.initial_seed() % 2 ** 32
        np.random.seed(randseed)
        random.seed(randseed)

    g = torch.Generator()
    g.manual_seed(randseed)

    if not normalize_data:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((image_size, image_size)),
            torchvision.transforms.ToTensor(),
            # transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
        ])
    else:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((image_size, image_size)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    root = f'./DomainNet/{domain}/{group}/{domain}'

    # torchvision.datasets.ImageFolder
    dataset = torchvision.datasets.ImageFolder(
        root = root,
        transform = transform,
    )
    print(len(dataset))

    dataloader = DataLoader(dataset, batch_size=batch_size,
                            worker_init_fn=seed_worker,
                            generator=g,
                            shuffle=True,
                            pin_memory=pin_memory,
                            num_workers=num_workers)

    return {'train': dataloader}, list(dataset.class_to_idx.keys())

In [30]:
num_samples = 5
for domain in ['real']: # 'painting',
    for group in ['mammals']:
        for image_size in [32, 64, 128, 224]:
            full_dataloaders, classes_str = get_domain_net(f'{domain}', group, False, image_size = image_size, batch_size=256, pin_memory=True, num_workers=2)
            print(classes_str)
            samples = get_samples(full_dataloaders['train'], num_samples, images_are_normalized=False, num_classes=len(classes_str))
            plot_samples(samples, classes_str, images_are_normalized=False, path=f'DomainNet/{group}/Samples domain_{domain} - img-size_{image_size}')

15825
['bear', 'beard', 'camel', 'cat', 'cow', 'dog', 'elephant', 'giraffe', 'hedgehog', 'horse', 'hot_dog', 'kangaroo', 'lion', 'monkey', 'mouse', 'panda', 'pig', 'rabbit', 'raccoon', 'rhinoceros', 'sheep', 'squirrel', 'teddy-bear', 'tiger', 'zebra']


  3%|▎         | 2/62 [00:07<03:39,  3.66s/it]


15825
['bear', 'beard', 'camel', 'cat', 'cow', 'dog', 'elephant', 'giraffe', 'hedgehog', 'horse', 'hot_dog', 'kangaroo', 'lion', 'monkey', 'mouse', 'panda', 'pig', 'rabbit', 'raccoon', 'rhinoceros', 'sheep', 'squirrel', 'teddy-bear', 'tiger', 'zebra']


  3%|▎         | 2/62 [00:06<03:08,  3.14s/it]


15825
['bear', 'beard', 'camel', 'cat', 'cow', 'dog', 'elephant', 'giraffe', 'hedgehog', 'horse', 'hot_dog', 'kangaroo', 'lion', 'monkey', 'mouse', 'panda', 'pig', 'rabbit', 'raccoon', 'rhinoceros', 'sheep', 'squirrel', 'teddy-bear', 'tiger', 'zebra']


  3%|▎         | 2/62 [00:08<04:01,  4.03s/it]


15825
['bear', 'beard', 'camel', 'cat', 'cow', 'dog', 'elephant', 'giraffe', 'hedgehog', 'horse', 'hot_dog', 'kangaroo', 'lion', 'monkey', 'mouse', 'panda', 'pig', 'rabbit', 'raccoon', 'rhinoceros', 'sheep', 'squirrel', 'teddy-bear', 'tiger', 'zebra']


  3%|▎         | 2/62 [00:09<04:30,  4.51s/it]


<Figure size 640x480 with 0 Axes>